# Transformer

## Data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from collections import Counter

import math
import numpy as np
import re
from tqdm import tqdm
import requests
import os

In [ ]:
url = "https://www.gutenberg.org/files/100/100-0.txt"
response = requests.get(url)
shakespeare_text = response.text

shakespeare_text = re.sub(r'\s+', ' ', shakespeare_text)
hamlet_text = shakespeare_text[927066:1105544]

In [ ]:
def tokenize(text):
    return text.lower().split()

vocab = Counter(tokenize(hamlet_text))
vocab = {word: i+2 for i, (word, _) in enumerate(vocab.items())}
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1

def encode(text, vocab):
    return [vocab.get(token, vocab['<UNK>']) for token in tokenize(text)]

In [ ]:
class TextDataset(Dataset):
    def __init__(self, text, vocab, seq_length):
        self.sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
        self.vocab = vocab
        self.seq_length = seq_length
        self.encoded_sentences = [encode(sentence, vocab) for sentence in self.sentences]
        self.encoded_sentences = [sent for sent in self.encoded_sentences if len(sent) > 1]

    def __len__(self):
        return len(self.encoded_sentences)

    def __getitem__(self, idx):
        sentence = self.encoded_sentences[idx]
        input_seq = sentence[:self.seq_length]
        target_seq = sentence[1:self.seq_length + 1]

        if len(input_seq) < self.seq_length:
            input_seq.extend([self.vocab['<PAD>']] * (self.seq_length - len(input_seq)))
            target_seq.extend([self.vocab['<PAD>']] * (self.seq_length - len(target_seq)))

        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_seq, dtype=torch.long)

In [ ]:
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=vocab['<PAD>'])
    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=vocab['<PAD>'])
    return inputs, targets

## Model

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.fc = nn.Linear(d_model, d_model)
        
    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)
        
        q = self.query(q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = self.key(k).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = self.value(v).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attn = F.softmax(scores, dim=-1)
        context = torch.matmul(attn, v)
        
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.fc(context)
        
        return output

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.fc2(self.dropout(F.relu(self.fc1(x))))

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.layernorm1(x + self.dropout(attn_output))
        
        ff_output = self.feed_forward(x)
        x = self.layernorm2(x + self.dropout(ff_output))
        
        return x

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.layernorm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        self_attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.layernorm1(x + self.dropout(self_attn_output))
        
        cross_attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.layernorm2(x + self.dropout(cross_attn_output))
        
        ff_output = self.feed_forward(x)
        x = self.layernorm3(x + self.dropout(ff_output))
        
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, d_ff=2048, num_layers=6, dropout=0.1):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src = self.encoder_embedding(src)
        src = self.positional_encoding(src)
        
        for layer in self.encoder_layers:
            src = layer(src, src_mask)
        
        tgt = self.decoder_embedding(tgt)
        tgt = self.positional_encoding(tgt)
        
        for layer in self.decoder_layers:
            tgt = layer(tgt, src, src_mask, tgt_mask)
        
        output = self.fc_out(tgt)
        
        return output

In [ ]:
def average_sentence_length(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    total_words = sum(len(tokenize(sentence)) for sentence in sentences)
    average_length = total_words / len(sentences)
    return average_length

average_length = average_sentence_length(hamlet_text)
print(f'Average number of words per sentence: {average_length:.2f}')

In [ ]:
# Hyperparameters
seq_length = 10
batch_size = 512
embedding_dim = 128
hidden_size = 256
learning_rate = 0.001
epochs = 1

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = Transformer(len(vocab), len(vocab)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

dataset = TextDataset(hamlet_text, vocab, seq_length)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
def train_and_test(model, train_loader, test_loader, criterion, optimizer, epochs, device):
    model.to(device)
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, inputs)
            loss = criterion(outputs.view(-1, outputs.size(2)), targets.view(-1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        avg_train_loss = train_loss / len(train_loader)
        
        # Testing phase
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for inputs, targets in tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Testing]", leave=False):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs, inputs)
                loss = criterion(outputs.view(-1, outputs.size(2)), targets.view(-1))
                test_loss += loss.item()
        avg_test_loss = test_loss / len(test_loader)

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')

# Train the model
train_and_test(model, train_loader, test_loader, criterion, optimizer, epochs, device)

In [ ]:
# Prediction function
def predict(model, start_sequence, vocab, seq_length, max_length=100):
    model.eval()
    tokens = tokenize(start_sequence)
    generated = tokens
    for _ in range(max_length):
        input_seq = torch.tensor([vocab.get(token, vocab['<UNK>']) for token in generated[-seq_length:]], dtype=torch.long).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_seq, input_seq)
        
        next_token = output[:, -1, :].argmax(dim=-1).item()
        
        next_word = [word for word, idx in vocab.items() if idx == next_token][0]
        generated.append(next_word)
        
        if next_word == '<PAD>':
            break

    return ' '.join(generated)

input_text = "to be or not to be that is the question"
predicted_output = predict(model, input_text, vocab, seq_length)
print(predicted_output)